In [ ]:
!mkdir content

In [ ]:
!unzip -q /content/archive.zip -d ./content/ckplus

In [ ]:
!ls ./content/ckplus

anger  contempt  disgust  fear	happy  sadness	surprise


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models

In [ ]:
def load_images(directory):
    images = []
    labels = []
    for subdir in os.listdir(directory):
        label = subdir
        for file in os.listdir(os.path.join(directory, subdir)):
            img_path = os.path.join(directory, subdir, file)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
            image = cv2.resize(image, (48, 48))  # Resize image to 48x48
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
def preprocess_data(images, labels):
    # Normalize pixel values
    images = images / 255.0
    # Split dataset into training and testing sets
    train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
    return train_images, test_images, train_labels, test_labels

In [ ]:
def encode_labels(train_labels, test_labels):
    label_encoder = LabelEncoder()
    train_labels_encoded = label_encoder.fit_transform(train_labels)
    test_labels_encoded = label_encoder.transform(test_labels)
    return train_labels_encoded, test_labels_encoded

In [ ]:
# Step 6: Build the model
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
# Step 7: Load and preprocess images
directory = "./content/ckplus"  # Update if necessary
images, labels = load_images(directory)

In [ ]:
# Step 8: Split the dataset
train_images, test_images, train_labels, test_labels = preprocess_data(images, labels)

In [ ]:
# Step 9: Prepare labels
train_labels_encoded, test_labels_encoded = encode_labels(train_labels, test_labels)

In [ ]:
# Step 10: Build the model
input_shape = (48, 48, 1)  # Image dimensions
num_classes = len(np.unique(labels))  # Number of unique emotion classes
model = build_model(input_shape, num_classes)

In [ ]:
# Step 11: Train the model
history = model.fit(train_images, train_labels_encoded, epochs=10, batch_size=32, validation_data=(test_images, test_labels_encoded))

Epoch 1/10
25/25 [==============================] - 5s 149ms/step - loss: 1.8394 - accuracy: 0.2347 - val_loss: 1.7901 - val_accuracy: 0.3706
Epoch 2/10
25/25 [==============================] - 3s 104ms/step - loss: 1.6571 - accuracy: 0.4031 - val_loss: 1.3638 - val_accuracy: 0.6041
Epoch 3/10
25/25 [==============================] - 2s 95ms/step - loss: 1.1528 - accuracy: 0.6429 - val_loss: 0.8503 - val_accuracy: 0.6751
Epoch 4/10
25/25 [==============================] - 2s 96ms/step - loss: 0.6872 - accuracy: 0.7589 - val_loss: 0.6380 - val_accuracy: 0.7766
Epoch 5/10
25/25 [==============================] - 3s 141ms/step - loss: 0.4799 - accuracy: 0.8342 - val_loss: 0.4434 - val_accuracy: 0.8477
Epoch 6/10
25/25 [==============================] - 3s 101ms/step - loss: 0.3513 - accuracy: 0.8801 - val_loss: 0.3712 - val_accuracy: 0.9036
Epoch 7/10
25/25 [==============================] - 2s 96ms/step - loss: 0.2286 - accuracy: 0.9133 - val_loss: 0.3559 - val_accuracy: 0.8832
Epoch 8/1

In [ ]:
# Step 12: Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels_encoded)
print('Test accuracy:', test_acc)

7/7 [==============================] - 0s 21ms/step - loss: 0.2738 - accuracy: 0.9188
Test accuracy: 0.9187816977500916


In [ ]:
# Save the trained model
model.save("/content/trained_model.h5")